In [ ]:
from utils_others import *

In [ ]:
genai.configure(api_key="", transport='rest')
# 放入gemini api (我的api余额已经快用光光了.....捂脸)

In [ ]:
# 模型设定
gemini_hypers = {
    # temp=[0.2, 0.4, 0.6, 0.8, 1.0],
    'temp': 0.2,
    'alpha': 0.95,
    'beta': 0.3,
    'basic': [False],
    'settings': [SerializerSettings(base=10, prec=3, signed=True,half_bin_correction=True)],
    # prec可能=3
}

model_hypers = {
    'gemini-1.0-pro': {'model': 'gemini-1.0-pro', **gemini_hypers},
    # 'gemini-pro': {'model': 'gemini-pro', **gemini_hypers}
}

model_predict_fns = {
    'gemini-1.0-pro': get_llmtime_predictions_data,  # 此处假定用gpt系列的encoding方式来进行预测
    # 'gemini-pro': get_llmtime_predictions_data
}

In [ ]:
message = ("You are a helpful assistant that specializing in time series analysis. The user will provide a sequence. The sequence is represented by decimal strings separated by commas. You need to infer the most probable underlying cycle of the sequence, even though there may also be a trend in this sequence. Do not assume that the underlying cycle has to stick to the popular cycles like 7 (days in a week), 12 (months in a year) and 30 (days in a month), just infer the cycle fully based on the inherent cycle of the given sequence. The underlying cycle of the sequence is strictly below 14. Please infer the cycle without producing any additional text. Do not say anything like 'the underlying cycle is', just return the number. Sequence:\n")

In [ ]:
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_NONE"
  },
]

In [ ]:
# 一些说明:
# 1. 周期不是时间序列客观存在的，而是人为赋予的概念 (STL 里也提到是 known before), 因此这里限制周期在真实值上下一个周期长度内, 便于观察
# 2. 强制性使得大模型不关注常见的周期, 而只关注数据的真实周期--但实际这么做可能OOD
# 3. 在预测序列一开始加上 ',', 模型效果似乎会好些, 这和 prompt 写法有关...... 
# 4. 下面不是真实输入数据, 而是经过 Timellm 原始论文里 renormalization 后的输入_至于是否真实数据会表现得还不错......没做过实验,还不知道(抱头)

# Ausbeer: period = 4
# input_str = ',402,249,279,453,354,281,298,479,376,292,300,464,352,279,328,466,406,279,350,460,425,292,343,520,391,328,371,535,423,339,389,572,464,378,436,587,503,410,449,621,511,410,453,655,550,470,490,662,636,477,494,742,615,505,567,750,623,559,595,793,673,587,606,838,692,604,636,879,776,623,709,1006,791,748,759,987,866,763,726,983,888,722,765,970,836,765,774,1008,899,789,718,1056,873,744,754,985,894,709,808,922,970,737,800,1028,851,722,823,1030,813,662,726,942,765,716,688,978,789,688,701,983,778,711,714,940,825,686,737,948,810,737,752,1077,795,735,750,1011,834,739,714,1079,789,703,729,1026,744,673,694,935,722,696,673,892,757,610,701,933,707,668,686,909,671,647,647,881,720,647,664,922,'

# Wine: period = 12
# input_str = ',294,352,471,387,399,442,575,606,512,564,716,823,290,397,471,520,452,547,681,789,505,552,738,831,359,394,490,600,558,546,709,741,618,596,796,883,358,472,596,534,674,610,670,869,633,584,822,991,382,546,677,551,720,586,754,1029,586,656,833,1059,414,537,706,557,762,668,647,957,617,777,961,1099,469,532,697,633,637,706,859,999,713,694,908,1154,519,449,585,771,636,615,947,647,717,795,953,1204,489,581,600,745,613,637,989,670,597,800,938,992,361,511,606,672,534,481,797,670,621,723,877,1146,277,381,668,930,559,554,746,668,667,662,926,1018,327,548,514,454,546,609,832,716,'

# TurkeyPower: period = 7
input_str = ',624,870,883,878,862,889,773,653,945,958,1000,999,918,805,657,936,1026,979,956,986,888,731,855,844,809,839,840,815,709,934,961,933,954,910,699,571,860,946,921,867,768,646,576,873,822,792,802,840,681,528,776,823,813,912,885,711,606,888,915,950,890,836,730,604,862,865,879,883,849,736,608,876,840,877,902,907,773,609,878,885,875,836,780,672,453,712,762,690,732,657,565,451,696,744,689,768,706,585,460,804,755,734,719,705,627,607,783,810,734,745,724,589,512,728,751,744,738,724,621,367,239,295,354,600,615,564,430,699,677,721,700,708,658,524,716,723,720,647,671,597,509,735,744,770,803,815,716,581,845,851,852,860,835,732,579,837,825,809,813,821,703,524,813,832,812,816,809,716,556,839,826,866,879,862,735,565,860,869,866,896,897,776,600,878,887,895,875,547,379,397,446,491,688,708,681,696,631,935,927,930,942,941,831,666,951,1000,999,1007,1014,881,723,1022,1020,1028,1039,1032,902,718,1018,1024,1024,994,991,868,706,936,970,983,1023,1012,878,726,1038,1023,991,958,940,823,671,923,854,973,980,986,834,653,914,858,847,848,849,748,619,861,834,804,816,847,743,579,827,823,803,776,736,633,461,724,760,781,800,814,671,546,758,765,753,753,765,658,476,718,713,745,724,738,647,513,740,732,722,728,698,584,458,685,680,705,717,607,499,441,713,693,703,706,707,596,491,766,748,743,729,728,613,468,740,749,780,788,787,640,540,769,741,769,801,830,701,562,836,823,823,808'

In [ ]:

# if model in ['gemini-1.0-pro', 'gemini-pro']:  # 此处为主要修改位置
model = 'gemini-pro'
content_fin = []
# print(gemini_sys_message+extra_input+input_str)
model = genai.GenerativeModel(model)
num_samples = 20

for i in range(num_samples):
    print("Index:", i)
    response = model.generate_content(
        contents=message+input_str,  # 此处因为只有一个 contents 选项, 所以将所有的话语都放在一起
        generation_config=genai.types.GenerationConfig(
            temperature=0.4,
        ),
        safety_settings=safety_settings
    )
    tmp = response.text
    content_fin.append(tmp)


In [ ]:
content_fin